In [2]:
from collections import namedtuple
import numpy as np
import math
import random
import pandas as pd



In [2]:
Graph = namedtuple("Graph", ["nodes", "edges"])

nodes = ["A","B","C","D"]
edges = [
    ("A","B"),
    ("A","B"),
    ("A","C"),
    ("A","C"),
    ("A","D"),
    ("B","D"),
    ("C","D")
]

G = Graph(nodes,edges) 

adjacency_dict = {node:[] for node in G.nodes}

for edge in G.edges:
    node1, node2 = edge[0], edge[1]
    adjacency_dict[node1].append(node2)
    adjacency_dict[node2].append(node1)

adjacency_dict

{'A': ['B', 'B', 'C', 'C', 'D'],
 'B': ['A', 'A', 'D'],
 'C': ['A', 'A', 'D'],
 'D': ['A', 'B', 'C']}

In [3]:
[[5 for i in range(3)] for j in range(5)]

[[5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5]]

In [3]:
nodes = range(4)
edges = [
    (0,1),
    (0,1),
    (0,2),
    (0,2),
    (0,3),
    (1,3),
    (2,3)
]

G = Graph(nodes,edges)

adj = [[0 for node in G.nodes] for node in G.nodes] 

for edge in G.edges:
    node1,node2 = edge[0], edge[1]
    adj[node1][node2] += 1
    adj[node2][node1] += 1
adj

[[0, 2, 2, 1], [2, 0, 0, 1], [2, 0, 0, 1], [1, 1, 1, 0]]

<h1>Generating an adjacency matrix for a graph</h1>



<p>We define the graph nodes and edges as follows:</p>
<img src="graphs.png" height="400" >
<p> Then we generate a graph object using namedtuple from the collections module. </p>
<p>We then define the edges manually to then generate an adjacency matrix</p>

<h3>Define the graph</h3>

In [5]:
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,1),
    (0,5),
    (0,6),
    (1,0),
    (1,2),
    (1,6),
    (2,1),
    (2,3),
    (2,6),
    (3,2),
    (3,4),
    (3,6),
    (4,3),
    (4,5),
    (4,6),
    (5,4),
    (5,6),
    (5,0),
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [4]:
numberofNodes = len(G.nodes)
matrix_shape = (numberofNodes, numberofNodes)
adj_matrix = np.zeros(matrix_shape)
for edge in G.edges:
    node1, node2 = edge[0], edge[1]
    adj_matrix[node1][node2] +=1
    adj_matrix[node2][node1] +=1

adj_matrix = adj_matrix/2 #just to make everything 1s and 0s



<h1>Creating transition matrices</h1>

      
<p>The following adjacency matrix was generated by the above code for the graph(see below):</p>
<img src="graphs.png" height="400" >
      
         0   1   2   3   4   5   6
      0 [0., 1., 0., 0., 0., 1., 1.]
      1 [1., 0., 1., 0., 0., 0., 1.]
      2 [0., 1., 0., 1., 0., 0., 1.]
      3 [0., 0., 1., 0., 1., 0., 1.]
      4 [0., 0., 0., 1., 0., 1., 1.]
      5 [1., 0., 0., 0., 1., 0., 1.]
      6 [1., 1., 1., 1., 1., 1., 0.]

<p>Using this adjacency matrix, we can easily create the transition matrix based off the probabilities defined in the figure.</p>
<p>The stay probability of a node is defined by the looping arrow.</p>
   <p>This stay probability is simply the edge connecting the node to itself. This is represented by the <strong>diagonal</strong> of the adjacency matrix.</p>
<p>The transition probability of a node (to the next node) is simply then : (1 - stay probability of that node) / number of adjacent nodes.</p>

In [4]:
def generate_transitionMatrix(graph,stay_probability=0):
    """Function to create the transition matrix. We assume that the graph has already been created and the 
    relevant nodes and edges are clearly defined. """

    #get the number of nodes
    numberofNodes = len(graph.nodes)
    #get shape of the transition matrix 
    matrix_shape = (numberofNodes, numberofNodes)
    #create the adjacent matrix with zeros
    adj_matrix = np.zeros(matrix_shape)

    for edge in graph.edges:
        node1,node2 = edge[0],edge[1]
        adj_matrix[node1][node2] += 1
        adj_matrix[node2][node1] += 1
    adj_matrix = adj_matrix/2 #just to make everything 1s and 0s
    
    #lets keep the calculation of the transition probabilites dynamic based off the number of adjacent nodes from
    #a starting node and the stay probability of that starting node
    #therefore what we should have is: transition probability = (1 - stay probability) / number of adjacent nodes

    np.fill_diagonal(adj_matrix,stay_probability) #set the stay probability in the adj matrix for the for loop 
    transition_matrix = np.copy(adj_matrix)

    #lets get the number of neighbours for each node by reading off the total number of 1s (adjacent nodes)
    for index,row in enumerate(adj_matrix):
        #we assume that the loops of a node to itself are already defined in the edges of the graph
        number_adjacent_nodes = np.count_nonzero(row == 1)
        transition_probability = (1 - stay_probability) / (number_adjacent_nodes)

        #adjust the decimal places
        transition_probability

        #replace the 1s (adjacent nodes) with their transition probability
        transition_matrix[index] = np.where(row ==1, f"{transition_probability:.2f}", row) #formatting the transition probability to two decimal places

    #set the stay probability of the matrix for each node 
    np.fill_diagonal(transition_matrix,stay_probability)
    #replace the diagonals with zeros for the adjacent matrix
    np.fill_diagonal(adj_matrix, 0)

    return adj_matrix, transition_matrix
        



In [8]:
#define graph, nodes and edges
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,0), #loop back to itself
    (0,1),
    (0,5),
    (0,6),
    (1,1), #loop back to itself
    (1,0),
    (1,2),
    (1,6),
    (2,2), #loop back to itself
    (2,1),
    (2,3),
    (2,6),
    (3,3), #loop back to itself
    (3,2),
    (3,4),
    (3,6),
    (4,4), #loop back to itself
    (4,3),
    (4,5),
    (4,6),
    (5,5), #loop back to itself
    (5,4),
    (5,6),
    (5,0),
    (6,6), #loop back to itself
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [9]:
colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G, stay_probability=0.1)
colour_adjmatrix, colour_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.1 , 0.3 , 0.  , 0.  , 0.  , 0.3 , 0.3 ],
        [0.3 , 0.1 , 0.3 , 0.  , 0.  , 0.  , 0.3 ],
        [0.  , 0.3 , 0.1 , 0.3 , 0.  , 0.  , 0.3 ],
        [0.  , 0.  , 0.3 , 0.1 , 0.3 , 0.  , 0.3 ],
        [0.  , 0.  , 0.  , 0.3 , 0.1 , 0.3 , 0.3 ],
        [0.3 , 0.  , 0.  , 0.  , 0.3 , 0.1 , 0.3 ],
        [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.1 ]]))

In [10]:
texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G, stay_probability=0.3)
texture_adjmatrix, texture_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.3 , 0.23, 0.  , 0.  , 0.  , 0.23, 0.23],
        [0.23, 0.3 , 0.23, 0.  , 0.  , 0.  , 0.23],
        [0.  , 0.23, 0.3 , 0.23, 0.  , 0.  , 0.23],
        [0.  , 0.  , 0.23, 0.3 , 0.23, 0.  , 0.23],
        [0.  , 0.  , 0.  , 0.23, 0.3 , 0.23, 0.23],
        [0.23, 0.  , 0.  , 0.  , 0.23, 0.3 , 0.23],
        [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.3 ]]))

In [11]:
shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G, stay_probability=0.5)
shape_adjmatrix, shape_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.5 , 0.17, 0.  , 0.  , 0.  , 0.17, 0.17],
        [0.17, 0.5 , 0.17, 0.  , 0.  , 0.  , 0.17],
        [0.  , 0.17, 0.5 , 0.17, 0.  , 0.  , 0.17],
        [0.  , 0.  , 0.17, 0.5 , 0.17, 0.  , 0.17],
        [0.  , 0.  , 0.  , 0.17, 0.5 , 0.17, 0.17],
        [0.17, 0.  , 0.  , 0.  , 0.17, 0.5 , 0.17],
        [0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.5 ]]))

<h1>Creating Design Files - Pseudo Code</h1>
<p>Create an empty dataframe which should hold columns for</p>
    <ul>
    <li>Participant ID</li>
    <li>Block Number</li>
    <li>Trial Number</li>
    <li>Colour_stim</li>
    <li>Texture_stim</li>
    <li>Shape_stim</li>
    <li>Colour_choice</li>
    <li>Texture_choice</li>
    <li>Shape_stim</li>
    </ul>
<p>Set a variable for number of trials</p>
<p>Set a variable for number of participants for which we need to create design files</p>
<p>Set a flag for every 20th trial to trigger a choice trial.</p>
<p>Set up an array containing the three stay probabilities [0.1, 0.3, 0.5]. The stay probabilities need to be included as a parameter into the function to generate transition probabilities </p>
<p>Loop through the range of participants</p>
    <p>Pick a transition probability randomly for each of the dimensions. We do this to ensure participants  experience fast,medium and slow transitions of the dimensions </p>
    <p>Set up adjacency and transition matrices for each dimension using the functions with the chosen transition probs in the step above.</p>
    <p>Set up variables to hold current rows for colour, texture and shape</p>
    <p>Set up a variable for block, update this every 180 trials by +1. </p>
    <p>Loop through number of trials : </p>
        <ul>
        <li>Just for the first iteration, generate 3 random whole numbers between 1-7. These 3 numbers will be the index of the row of the transition matrix of each dimension. Record these 3 numbers as the current row</li>
        <li>These resulting numbers will be used to generate the first object (colour + texture + shape) </li>
        <li>From the second iteration onwards (for all the following shapes), generate 3 random numbers between 0 - 1.</li>
            <li>If the randomly generated number for all or any dimension, is greater than the stay probability of that dimension, we transition to the adjacent node. To transition, we do the following:</li>
            <li>To determine which node (so basically decide the next row) to transition to based off the current row, we look for adjacent nodes (columns) in the current row of the matrix where != 0.</li>
            <li>Note that sometimes the stay probabilities are greater than the transition probabilities.</li>
            <li> If the current row != 6 (7th row), then we follow the steps below to find the node to transition-</li>
            <li>If difference <= 0.4, transition to first neighbour, get column index, this number/index becomes current row. Update the respective dimension column(s) for that trial with the row number</li>
            <li>If difference >0.4 & <= 0.7, transition to second neighbour, get column index, this number/index becomes current row.</li>
            <li>If difference >0.7 , transition to third neighbour, get column index, this number/index becomes current row.</li>
            <li>If row == 6, then we randomly pick a neighbour to transition to.</li>
            <li>Every 20th trial, or rather If the trial%20 == 0, then we generate 3 random whole numbers for the choice columns for the choice trials  </li>
            <li>If the randomly generated number for all or any dimension, is less than stay probability, we don't transition. Therefore current row remains the same</li>
            <li>To include our choice trials, if trial % 20, we draw also draw 3 random numbers and add it to the colour_choice, texture_choice and shape_choice rows of the dataframe. Otherwise they remain NaNs</li>
            </ul>

In [12]:
import pandas as pd
import numpy as np
import random

In [6]:
#define graph, nodes and edges
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,0), #loop back to itself
    (0,1),
    (0,5),
    (0,6),
    (1,1), #loop back to itself
    (1,0),
    (1,2),
    (1,6),
    (2,2), #loop back to itself
    (2,1),
    (2,3),
    (2,6),
    (3,3), #loop back to itself
    (3,2),
    (3,4),
    (3,6),
    (4,4), #loop back to itself
    (4,3),
    (4,5),
    (4,6),
    (5,5), #loop back to itself
    (5,4),
    (5,6),
    (5,0),
    (6,6), #loop back to itself
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [15]:
def generateChoiceTrials(total_trials, n_choice_trials, start_trial, jitter_range):
    """Function generates an array containing choice trials for the participants. The start trial parameter is used to
    to define the first choice trial. The total trials are of the experiment. The parameter
    n_choicetrials is the number of choice trials the participant gets. We define the starting trial and a jitter range.
    The minimum interval between trials are calculated inside the function and a random jitter from 0 to 5 is added
    to the interval. This makes the trials less predictable"""

    #generate the random choice trials between 1 and 240 trials with the first choice trial appearing at trial 10 (start trial) or later
    #lets start with generating random trial numbers which are greater than 10 but less than 20
    #we start with a first trial being a number between 10 and 15
    # subsequent trials are generated if trial is greater than the previous trial and if the trial is between 10 and 20 increments from the previous trial
    #these trials must be a total of 18 trials and should be spaced out so they fill the span of 240 total trials

    minimum_interval = total_trials // n_choice_trials
    choice_trials = []


    for i in range(n_choice_trials):
        base = start_trial + i * minimum_interval
        jitter = random.randint(0, jitter_range)
        subsequent_trial = min(base + jitter, total_trials - 1)
        choice_trials.append(subsequent_trial)

    return choice_trials

a = generateChoiceTrials(200*8,18*8,10,5)    
a

[14,
 24,
 35,
 46,
 58,
 69,
 77,
 87,
 102,
 109,
 124,
 131,
 146,
 156,
 167,
 180,
 187,
 202,
 213,
 223,
 232,
 244,
 252,
 264,
 274,
 285,
 297,
 308,
 320,
 333,
 345,
 356,
 362,
 373,
 388,
 397,
 411,
 420,
 428,
 444,
 454,
 462,
 475,
 483,
 498,
 505,
 521,
 527,
 543,
 549,
 563,
 574,
 585,
 593,
 606,
 620,
 630,
 639,
 650,
 660,
 671,
 686,
 695,
 708,
 719,
 727,
 741,
 752,
 761,
 774,
 783,
 791,
 806,
 818,
 828,
 838,
 851,
 858,
 868,
 884,
 894,
 902,
 914,
 925,
 938,
 948,
 960,
 971,
 981,
 990,
 1004,
 1015,
 1025,
 1038,
 1046,
 1056,
 1066,
 1080,
 1088,
 1103,
 1112,
 1124,
 1132,
 1147,
 1155,
 1170,
 1181,
 1187,
 1202,
 1213,
 1224,
 1231,
 1246,
 1257,
 1265,
 1280,
 1288,
 1302,
 1312,
 1321,
 1330,
 1344,
 1353,
 1368,
 1376,
 1390,
 1397,
 1410,
 1419,
 1432,
 1441,
 1455,
 1462,
 1478,
 1486,
 1498,
 1510,
 1518,
 1533,
 1540,
 1550,
 1563,
 1576,
 1587]

Loop through all trials
find potential indices from trial 10 onwards where the distractor were not consistent across all dimensions within the 200 trials
record these indices
record the distractors as well for that indice
from the record of these indices, choose 18 of them each with a gap of 8-12 trials apart
repeat for other blocks




In [ ]:
master_df = []

nTrials = 200
nParticipants = 50
nBlocks = 8
nSessions = 2
stayProbs = [0.1,0.3,0.5]
block = 1 #starting with just the first block
# this map is to check if the randomly generated nodes for a distractor in a choice trial are consistent with
    # the preceeding stimulus object in the graph structure. If the preceeding stimulus object has the nodes 1 1 1 (Colour,Texture,Shape)
    # and the distractor has the nodes 2 2 4 (Colour, Texture, Shape), then its nodes are consistent with the graph structure
    # as these nodes are neighbours. Therefore we populate the distractor_consistent_colour, distractor_consistent_texture with 1s and 0 for distractor_consistent_shape
possibleNeighbours = {
    0 : [0,1,5,6],
    1 : [1,0,2,6],
    2 : [2,1,3,6],
    3 : [3,2,4,6],
    4 : [4,3,5,6],
    5 : [5,4,0,6],
    6 : [0,1,2,3,4,5,6]
}




for participant in range(nParticipants):
    
    #create dataframe for design file for every participant
    #regular columns
    columns = ["Participant_ID",
    "Block Number",
    "Session Number",
    "Trial Number",
    "Stay_Probability(Colour)",
    "Stay_Probability(Texture)",
    "Stay_Probability(Shape)",
    "Slow_stim", #shows the node of the slow dimension (e.g.: if colour is the slower changing dimension, the node will be shown here. This is the same as the value in "Colour_stim")
    "Medium_stim",
    "Fast_stim",
    "Colour_stim",
    "Texture_stim",
    "Shape_stim",
    "Choice Trial Index",
    "Colour_distractor",
    "Texture_distractor",
    "Shape_distractor",
    "Slow_distractor", #shows the node of the distractor in terms of the slow dimension (e.g.: if colour is the slower changing dimension for the stimulus, the node for colour of the distractor will be shown here. This is the same as the value in "Colour_distractor")
    "Medium_distractor",
    "Fast_distractor",
    "Distractor_consistent_colour",
    "Distractor_consistent_texture",
    "Distractor_consistent_shape",
    "Distractor_consistent_slow",
    "Distractor_consistent_medium",
    "Distractor_consistent_fast",
    "Distractor_shares_colour", #column with 1s and 0s to show if the randomly generated distractor has the same dimensions as the stimulus
    "Distractor_shares_texture",
    "Distractor_shares_shape",
    "Distractor_shares_slow", #column with 1s and 0s to show if the randomly generated distractor shows the same dimensions as the stimulus, however just mapped based off the speed 
    "Distractor_shares_medium",
    "Distractor_shares_fast"
    ]


    #prepare the stayprobabilities and transition matrices for each dimension
    random.shuffle(stayProbs)
    #assign the respective dimensions to the speeds in which they transition
    slow = np.argmax(stayProbs)
    medium = np.where(np.array(stayProbs) ==0.3)[0][0]
    fast = np.argmin(stayProbs)

    # the speed map is to just map the nodes on to the dimension, based off the speed in which it changes
    # e.g. if colour has a stay prob of 0.1, this dimension changes faster, so a node of 2 in the Colour_stim column
    # would be mapped to the Stim_fast column
    dimensionSpeedMap = {
        0 : "Colour",
        1 : "Texture",
        2 : "Shape"
    }

    dimensionSpeeds = {
        "Slow": dimensionSpeedMap[slow] ,
        "Medium": dimensionSpeedMap[medium],
        "Fast": dimensionSpeedMap[fast]
    }

    colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[0])
    texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[1])
    shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[2])

    


    for session in range(nSessions):
        df = pd.DataFrame(columns=columns)
        rowIndex = 0
        for block in range(nBlocks):            
            currentRow_colour = None
            currentRow_shape = None
            currentRow_texture = None
            choiceTrialIndex = 1
            
            for trial in range(nTrials):
                df.loc[rowIndex,"Block Number"] = block + 1
                df.loc[rowIndex, "Session Number"] = session + 1
                df.loc[rowIndex,"Trial Number"] = trial + 1
                df.loc[rowIndex,"Stay_Probability(Colour)"] = stayProbs[0]
                df.loc[rowIndex,"Stay_Probability(Texture)"] = stayProbs[1]
                df.loc[rowIndex,"Stay_Probability(Shape)"] = stayProbs[2]

                stimNodeDimensionSpeedMap = {
                    "Colour": None,
                    "Texture": None,
                    "Shape": None
                }
            
                if trial == 0: #for the very first trial generate 3 random numbers (integers)
                    color_int = random.randint(0,6)
                    texture_int = random.randint(0,6)
                    shape_int = random.randint(0,6)

                    df.loc[rowIndex, "Colour_stim"] = color_int
                    df.loc[rowIndex, "Texture_stim"] = texture_int
                    df.loc[rowIndex, "Shape_stim"] = shape_int

                    currentRow_colour = color_int
                    currentRow_texture = texture_int
                    currentRow_shape = shape_int

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]


                else:
                    color_int = float(f"{random.uniform(0,1):.2f}")
                    texture_int = float(f"{random.uniform(0,1):.2f}")
                    shape_int = float(f"{random.uniform(0,1):.2f}")

                    ##For the Colour dimension
                    
                    if color_int > stayProbs[0]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number is greater than stay probability, we can transition")

                        colour_node = colour_transition_matrix[currentRow_colour] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((colour_node != 0) & (colour_node != stayProbs[0]))
                        adj_node_transprobs = [colour_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {colour_node}, Stay probability : {stayProbs[0]}, Random Number Drawn:{color_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")
                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_colour == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour
                        
                        else:
                            if color_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            elif color_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                        

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number: {color_int} is lower than stay probability {stayProbs[0]}, we stay", "\n")
                        df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                    ##For the texture dimension

                    if texture_int > stayProbs[1]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number is greater than stay probability, we can transition")
                        texture_node = texture_transition_matrix[currentRow_texture] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((texture_node != 0) & (texture_node != stayProbs[1]))
                        adj_node_transprobs = [texture_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {texture_node}, Stay probability : {stayProbs[1]}, Random Number Drawn:{texture_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_texture == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture
                        
                        else:
                            if texture_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")

                            elif texture_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number: {texture_int} is lower than stay probability {stayProbs[1]}, we stay", "\n")
                        df.loc[rowIndex, "Texture_stim"] = currentRow_texture


                    ##For the shape dimension
                    
                    if shape_int > stayProbs[2]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number is greater than stay probability, we can transition")
                        shape_node = shape_transition_matrix[currentRow_shape] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((shape_node != 0) & (shape_node != stayProbs[2]))
                        adj_node_transprobs = [shape_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {shape_node}, Stay probability : {stayProbs[2]}, Random Number Drawn:{shape_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_shape == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape
                        
                        else:
                            if shape_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {shape_node}","\n")

                            elif shape_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number: {shape_int} is lower than stay probability {stayProbs[2]}, we stay", "\n")
                        df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]
                
                rowIndex += 1 

            rowIndex_start_of_block = rowIndex - nTrials
            eligible_trials = []
            nodes_preced = None

            for trial in range(10, nTrials):  # start at trial 10
                rowIndex_current = rowIndex_start_of_block + trial
                
                c_prev = df.loc[rowIndex_current-1, "Colour_stim"]
                t_prev = df.loc[rowIndex_current-1, "Texture_stim"]
                s_prev = df.loc[rowIndex_current-1, "Shape_stim"]
                nodes_preced = [c_prev,t_prev,s_prev]

                distractor_colour = random.randint(0,6)
                distractor_texture = random.randint(0,6)
                distractor_shape = random.randint(0,6)
                distractorNodeVals = [distractor_colour,distractor_texture,distractor_shape]

                allMatch = (
                    distractor_colour in possibleNeighbours[c_prev],
                    distractor_texture in possibleNeighbours[t_prev],
                    distractor_shape in possibleNeighbours[s_prev]
                )

                if sum(allMatch) < 3:  # eligible trial
                    eligible_trials.append((trial, distractor_colour, distractor_texture, distractor_shape))
            
            #print(eligible_trials)

            choice_trials = []
            last_choice = -999
            for trial, dcol, dtex, dsha in eligible_trials:
                if trial - last_choice >= random.randint(8, 12):
                    choice_trials.append((trial, dcol, dtex, dsha))
                    last_choice = trial
                    if len(choice_trials) == 18:
                        break
            #print(len(choice_trials))


            for idx, (trial, dcol, dtex, dsha) in enumerate(choice_trials, start=1):
                rowIndex_current = rowIndex_start_of_block + trial
                df.loc[rowIndex_current, "Colour_distractor"] = dcol
                df.loc[rowIndex_current, "Texture_distractor"] = dtex
                df.loc[rowIndex_current, "Shape_distractor"] = dsha
                df.loc[rowIndex_current, "Choice Trial Index"] = idx

                stimNodeVals = [
                df.loc[rowIndex_current, "Colour_stim"],
                df.loc[rowIndex_current, "Texture_stim"],
                df.loc[rowIndex_current, "Shape_stim"]
                ]
                colsToModify = ["Distractor_shares_colour", "Distractor_shares_texture","Distractor_shares_shape"]
                distractorNodeVals = [dcol, dtex, dsha]

                for j, val in enumerate(distractorNodeVals):
                    df.loc[rowIndex_current, colsToModify[j]] = 1 if val == stimNodeVals[j] else 0


                stimNodeDimensionSpeedMap = {
                    "Colour": stimNodeVals[0],
                    "Texture": stimNodeVals[1],
                    "Shape": stimNodeVals[2]
                }
                distractorNodeDimensionsMap = {
                    "Colour": distractorNodeVals[0],
                    "Texture": distractorNodeVals[1],
                    "Shape": distractorNodeVals[2]
                }
                df.loc[rowIndex_current, "Slow_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Slow"]]
                df.loc[rowIndex_current, "Medium_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Medium"]]
                df.loc[rowIndex_current, "Fast_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]

                stimNodeSpeedVals = [
                    stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]],
                    stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]],
                    stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]
                ]
                distractorNodeSpeedVals = [
                    distractorNodeDimensionsMap[dimensionSpeeds["Slow"]],
                    distractorNodeDimensionsMap[dimensionSpeeds["Medium"]],
                    distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]
                ]
                speedColsToModify = ["Distractor_shares_slow", "Distractor_shares_medium", "Distractor_shares_fast"]

                for j, val in enumerate(distractorNodeSpeedVals):
                    df.loc[rowIndex_current, speedColsToModify[j]] = 1 if val == stimNodeSpeedVals[j] else 0

                consistentCols = ["Distractor_consistent_colour", "Distractor_consistent_texture", "Distractor_consistent_shape"]
                consistentSpeedCols = ["Distractor_consistent_slow", "Distractor_consistent_medium", "Distractor_consistent_fast"]

                nodes_preced = [
                    df.loc[rowIndex_current-1, "Colour_stim"],
                    df.loc[rowIndex_current-1, "Texture_stim"],
                    df.loc[rowIndex_current-1, "Shape_stim"]
                ]
                nodes_speed_preced = [
                    df.loc[rowIndex_current-1, "Slow_stim"],
                    df.loc[rowIndex_current-1, "Medium_stim"],
                    df.loc[rowIndex_current-1, "Fast_stim"]
                ]
                distractorSpeedVals = distractorNodeSpeedVals

                for j, val in enumerate(nodes_preced):
                    df.loc[rowIndex_current, consistentCols[j]] = 1 if distractorNodeVals[j] in possibleNeighbours[val] else 0

                for j, val in enumerate(nodes_speed_preced):
                    df.loc[rowIndex_current, consistentSpeedCols[j]] = 1 if distractorSpeedVals[j] in possibleNeighbours[val] else 0
                            

                    
        df.iloc[:,0] = participant + 1
        df.to_csv(f"../Experiment (No Backend)/public/DesignFiles/DesignFile_{participant+1}_session{session+1}.csv", index=False)
        #df.to_csv(f"DesignFile_{participant+1}_session{session+1}.csv", index=False)
                

                    

<h3>Copy of above script - adjusted to ensure that target is an entire transition from previous object. Distractor is also an entire transition from previous object and is max only consistent across 2 dimensions or less. </h3>

<p>Ensure also that last trial of every block is a choice trial</p>

In [ ]:
master_df = []

nTrials = 200
nParticipants = 1
nBlocks = 8
nSessions = 2
stayProbs = [0.1,0.3,0.5]
block = 1 #starting with just the first block
# this map is to check if the randomly generated nodes for a distractor in a choice trial are consistent with
    # the preceeding stimulus object in the graph structure. If the preceeding stimulus object has the nodes 1 1 1 (Colour,Texture,Shape)
    # and the distractor has the nodes 2 2 4 (Colour, Texture, Shape), then its nodes are consistent with the graph structure
    # as these nodes are neighbours. Therefore we populate the distractor_consistent_colour, distractor_consistent_texture with 1s and 0 for distractor_consistent_shape
possibleNeighbours = {
    0 : [0,1,5,6],
    1 : [1,0,2,6],
    2 : [2,1,3,6],
    3 : [3,2,4,6],
    4 : [4,3,5,6],
    5 : [5,4,0,6],
    6 : [0,1,2,3,4,5,6]
}




for participant in range(nParticipants):
    
    #create dataframe for design file for every participant
    #regular columns
    columns = ["Participant_ID",
    "Block Number",
    "Session Number",
    "Trial Number",
    "Stay_Probability(Colour)",
    "Stay_Probability(Texture)",
    "Stay_Probability(Shape)",
    "Slow_stim", #shows the node of the slow dimension (e.g.: if colour is the slower changing dimension, the node will be shown here. This is the same as the value in "Colour_stim")
    "Medium_stim",
    "Fast_stim",
    "Colour_stim",
    "Texture_stim",
    "Shape_stim",
    "Choice Trial Index",
    "Colour_distractor",
    "Texture_distractor",
    "Shape_distractor",
    "Slow_distractor", #shows the node of the distractor in terms of the slow dimension (e.g.: if colour is the slower changing dimension for the stimulus, the node for colour of the distractor will be shown here. This is the same as the value in "Colour_distractor")
    "Medium_distractor",
    "Fast_distractor",
    "Target_shares_colour",
    "Target_shares_texture",
    "Target_shares_shape",
    "Target_consistent_slow",
    "Target_consistent_medium",
    "Target_consistent_fast",
    "Distractor_consistent_colour",
    "Distractor_consistent_texture",
    "Distractor_consistent_shape",
    "Distractor_consistent_slow",
    "Distractor_consistent_medium",
    "Distractor_consistent_fast",
    "Distractor_shares_colour_target", #column with 1s and 0s to show if the randomly generated distractor has the same dimensions as the target
    "Distractor_shares_texture_target",
    "Distractor_shares_shape_target",
    "Distractor_shares_slow_target", #column with 1s and 0s to show if the randomly generated distractor shows the same dimensions as the target, however just mapped based off the speed 
    "Distractor_shares_medium_target",
    "Distractor_shares_fast_target",
    "Distractor_shares_colour_object", #column with 1s and 0s to show if the randomly generated distractor has the same dimensions as the target
    "Distractor_shares_texture_object",
    "Distractor_shares_shape_object",
    "Distractor_shares_slow_object", #column with 1s and 0s to show if the randomly generated distractor shows the same dimensions as the target, however just mapped based off the speed 
    "Distractor_shares_medium_object",
    "Distractor_shares_fast_object",
    
    ]


    #prepare the stayprobabilities and transition matrices for each dimension
    random.shuffle(stayProbs)
    #assign the respective dimensions to the speeds in which they transition
    slow = np.argmax(stayProbs)
    medium = np.where(np.array(stayProbs) ==0.3)[0][0]
    fast = np.argmin(stayProbs)

    # the speed map is to just map the nodes on to the dimension, based off the speed in which it changes
    # e.g. if colour has a stay prob of 0.1, this dimension changes faster, so a node of 2 in the Colour_stim column
    # would be mapped to the Stim_fast column
    dimensionSpeedMap = {
        0 : "Colour",
        1 : "Texture",
        2 : "Shape"
    }

    dimensionSpeeds = {
        "Slow": dimensionSpeedMap[slow] ,
        "Medium": dimensionSpeedMap[medium],
        "Fast": dimensionSpeedMap[fast]
    }

    colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[0])
    texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[1])
    shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[2])

    


    for session in range(nSessions):
        df = pd.DataFrame(columns=columns)
        rowIndex = 0
        for block in range(nBlocks):            
            currentRow_colour = None
            currentRow_shape = None
            currentRow_texture = None
            choiceTrialIndex = 1
            
            for trial in range(nTrials):
                df.loc[rowIndex,"Block Number"] = block + 1
                df.loc[rowIndex, "Session Number"] = session + 1
                df.loc[rowIndex,"Trial Number"] = trial + 1
                df.loc[rowIndex,"Stay_Probability(Colour)"] = stayProbs[0]
                df.loc[rowIndex,"Stay_Probability(Texture)"] = stayProbs[1]
                df.loc[rowIndex,"Stay_Probability(Shape)"] = stayProbs[2]

                stimNodeDimensionSpeedMap = {
                    "Colour": None,
                    "Texture": None,
                    "Shape": None
                }
            
                if trial == 0: #for the very first trial generate 3 random numbers (integers)
                    color_int = random.randint(0,6)
                    texture_int = random.randint(0,6)
                    shape_int = random.randint(0,6)

                    df.loc[rowIndex, "Colour_stim"] = color_int
                    df.loc[rowIndex, "Texture_stim"] = texture_int
                    df.loc[rowIndex, "Shape_stim"] = shape_int

                    currentRow_colour = color_int
                    currentRow_texture = texture_int
                    currentRow_shape = shape_int

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]


                else:
                    color_int = float(f"{random.uniform(0,1):.2f}")
                    texture_int = float(f"{random.uniform(0,1):.2f}")
                    shape_int = float(f"{random.uniform(0,1):.2f}")

                    ##For the Colour dimension
                    
                    if color_int > stayProbs[0]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number is greater than stay probability, we can transition")

                        colour_node = colour_transition_matrix[currentRow_colour] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((colour_node != 0) & (colour_node != stayProbs[0]))
                        adj_node_transprobs = [colour_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {colour_node}, Stay probability : {stayProbs[0]}, Random Number Drawn:{color_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")
                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_colour == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour
                        
                        else:
                            if color_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            elif color_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                        

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number: {color_int} is lower than stay probability {stayProbs[0]}, we stay", "\n")
                        df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                    ##For the texture dimension

                    if texture_int > stayProbs[1]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number is greater than stay probability, we can transition")
                        texture_node = texture_transition_matrix[currentRow_texture] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((texture_node != 0) & (texture_node != stayProbs[1]))
                        adj_node_transprobs = [texture_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {texture_node}, Stay probability : {stayProbs[1]}, Random Number Drawn:{texture_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_texture == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture
                        
                        else:
                            if texture_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")

                            elif texture_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number: {texture_int} is lower than stay probability {stayProbs[1]}, we stay", "\n")
                        df.loc[rowIndex, "Texture_stim"] = currentRow_texture


                    ##For the shape dimension
                    
                    if shape_int > stayProbs[2]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number is greater than stay probability, we can transition")
                        shape_node = shape_transition_matrix[currentRow_shape] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((shape_node != 0) & (shape_node != stayProbs[2]))
                        adj_node_transprobs = [shape_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {shape_node}, Stay probability : {stayProbs[2]}, Random Number Drawn:{shape_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_shape == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape
                        
                        else:
                            if shape_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {shape_node}","\n")

                            elif shape_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number: {shape_int} is lower than stay probability {stayProbs[2]}, we stay", "\n")
                        df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]
                
                rowIndex += 1 

            rowIndex_start_of_block = rowIndex - nTrials
            eligible_trials = []
            nodes_preced = None

            for trial in range(10, nTrials): 
                rowIndex_current = rowIndex_start_of_block + trial
                
                c_prev = df.loc[rowIndex_current-1, "Colour_stim"]
                t_prev = df.loc[rowIndex_current-1, "Texture_stim"]
                s_prev = df.loc[rowIndex_current-1, "Shape_stim"]
                nodes_preced = [c_prev,t_prev,s_prev]

                c_curr = df.loc[rowIndex_current, "Colour_stim"]
                t_curr = df.loc[rowIndex_current, "Texture_stim"]
                s_curr = df.loc[rowIndex_current, "Shape_stim"]

                full_transition = (
                    c_curr in possibleNeighbours[c_prev] and c_curr != c_prev and 
                    t_curr in possibleNeighbours[t_prev] and t_curr != t_prev and
                    s_curr in possibleNeighbours[s_prev] and s_curr != s_prev
                )

                distractor_colour = random.randint(0,6)
                distractor_texture = random.randint(0,6)
                distractor_shape = random.randint(0,6)
                distractorNodeVals = [distractor_colour,distractor_texture,distractor_shape]

                allMatch = (
                    distractor_colour in possibleNeighbours[c_prev] and distractor_colour!=c_prev,
                    distractor_texture in possibleNeighbours[t_prev] and distractor_texture!=t_prev,
                    distractor_shape in possibleNeighbours[s_prev] and distractor_shape!=s_prev
                )

                if sum(allMatch) < 3 and full_transition:  #append all the row indices where a trial is eligible
                    eligible_trials.append((trial, distractor_colour, distractor_texture, distractor_shape))
            

            num_choice_trials = 18
            mingap = 5
            maxgap = 10
            last_trial = nTrials - 1
            if last_trial not in [t[0]for t in eligible_trials]:
                last_trial = max([t[0] for t in eligible_trials if t[0]<nTrials])
            
            eligible_trials_before_last = [t for t in eligible_trials if t[0]!=last_trial]

            choice_trials = []
            current = eligible_trials_before_last[0][0]
            choice_trials.append(eligible_trials_before_last[0])

            while len(choice_trials) < num_choice_trials:
                candidates = [t for t in eligible_trials_before_last if t[0] > current + mingap]
                if not candidates:
                    remaining = [t for t in eligible_trials_before_last if t[0] > current]
                    if remaining:
                        current = remaining[0][0]
                        choice_trials.append(remaining[0])
                    break

                next_choice = candidates[0]
                for t in candidates:
                    if t[0]<=current + maxgap:
                        next_choice = t
                        break
                choice_trials.append(next_choice)
                current = next_choice[0]

            for t in eligible_trials:
                if t[0] == last_trial:
                    choice_trials.append(t)
                    break

            print(len(choice_trials))

            for index, (trial, d_col, d_text, d_shape) in enumerate(choice_trials, start=1):
                rowIndex_current = rowIndex_start_of_block + trial

                df.loc[rowIndex_current, "Colour_distractor"] = d_col
                df.loc[rowIndex_current, "Texture_distractor"] = d_text
                df.loc[rowIndex_current, "Shape_distractor"] = d_shape
                df.loc[rowIndex_current, "Choice Trial Index"] = index


                c_stim = df.loc[rowIndex_current, "Colour_stim"]
                t_stim = df.loc[rowIndex_current, "Texture_stim"]
                s_stim = df.loc[rowIndex_current, "Shape_stim"]


                stimNodeDimensionSpeedMap = {
                    "Colour": c_stim,
                    "Texture": t_stim,
                    "Shape": s_stim
                }
                distractorNodeDimensionsMap = {
                    "Colour": d_col,
                    "Texture": d_text,
                    "Shape": d_shape
                }
                df.loc[rowIndex_current, "Slow_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Slow"]]
                df.loc[rowIndex_current, "Medium_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Medium"]]
                df.loc[rowIndex_current, "Fast_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]


                for col, dim in zip(["Distractor_shares_colour_target", "Distractor_shares_texture_target","Distractor_shares_shape_target"], 
                    ["Colour", "Texture", "Shape"]):
                    df.loc[rowIndex_current, col] = 1 if distractorNodeDimensionsMap[dim] == stimNodeDimensionSpeedMap[dim] else 0
                for col,dim in zip(["Distractor_shares_slow_target", "Distractor_shares_medium_target", "Distractor_shares_fast_target"], 
                                   ["Slow", "Medium", "Fast"]):
                    df.loc[rowIndex_current, col] = 1 if distractorNodeDimensionsMap[dimensionSpeeds[dim]] == stimNodeDimensionSpeedMap[dimensionSpeeds[dim]] else 0




                #assign 1 if distractor is consistent on any dimension compared to the previous object, else 0
                prevRow = rowIndex_current - 1
                for col, stim_dim, dist_dim in zip(
                    ["Distractor_consistent_colour", "Distractor_consistent_texture", "Distractor_consistent_shape"]
                    ["Colour_stim", "Texture_stim", "Shape_stim"],
                    ["Colour_distractor", "Texture_distractor", "Shape_distractor"]
                ) :
                    df.loc[rowIndex_current,col] = 1 if df.loc[rowIndex_current, dist_dim] in possibleNeighbours[df.loc[prevRow, stim_dim]] else 0
                #assign 1 distractor is consistent on any dimension with regards to speed, compared to the previous object, else 0
                for col, stim_dim, dist_dim in zip(
                    ["Distractor_consistent_slow", "Distractor_consistent_medium", "Distractor_consistent_fast"],
                    ["Slow_stim", "Medium_stim", "Fast_stim"],
                    ["Slow_distractor", "Medium_distractor", "Fast_distractor"]
                ):
                    df.loc[rowIndex_current, col] = 1 if df.loc[rowIndex_current, dist_dim] in possibleNeighbours[df.loc[prevRow, stim_dim]] else 0
                            
                # #assign 1 if distractor shares the exact node/feature compared to the target, else 0 
                # for col, stim_dim, dist_dim in zip(
                #     ["Distractor_shares_colour_target", "Distractor_shares_texture_target", "Distractor_shares_shape_target"],
                #     ["Colour_stim", "Texture_stim", "Shape_stim"],
                #     ["Colour_distractor", "Texture_distractor", "Shape_distractor"]
                # ) :
                #     df.loc[rowIndex_current, col] = 1 if df.loc[rowIndex_current,dist_dim] == df.loc[rowIndex_current, stim_dim] else 0

                # #assign 1 if distractor shares the exact node/feature regarding speed compared to the target, else 0
                # for col, stim_dim, dist_dim in zip(
                #     ["Distractor_shares_slow_target", "Distractor_shares_medium_target", "Distractor_shares_fast_target"],
                #     ["Slow_stim", "Medium_stim", "Shape_stim"],
                #     ["Slow_distractor", "Medium_distractor", "Fast_distractor"]
                # ) :
                #     df.loc[rowIndex_current, col] = 1 if df.loc[rowIndex_current,dist_dim] == df.loc[rowIndex_current, stim_dim] else 0

                #assign 1 if distractor shares the exact node/feature compared to the previous object, else 0
                
                 
                    
        df.iloc[:,0] = participant + 1
        #df.to_csv(f"../Experiment (No Backend)/public/DesignFiles/DesignFile_{participant+1}_session{session+1}.csv", index=False)
        #df.to_csv(f"DesignFile_{participant+1}_session{session+1}.csv", index=False)
                

                    

19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19


<h3>Sanity Check Code</h3>

<p>Displaying the dataframe</p>

In [20]:
df

,Participant_ID,Block Number,Session Number,Trial Number,Stay_Probability(Colour),Stay_Probability(Texture),Stay_Probability(Shape),Slow_stim,Medium_stim,Fast_stim,...,Distractor_consistent_shape,Distractor_consistent_slow,Distractor_consistent_medium,Distractor_consistent_fast,Distractor_shares_colour,Distractor_shares_texture,Distractor_shares_shape,Distractor_shares_slow,Distractor_shares_medium,Distractor_shares_fast
0,50,1,2,1,0.5,0.1,0.3,3,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50,1,2,2,0.5,0.1,0.3,4,5,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50,1,2,3,0.5,0.1,0.3,4,5,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50,1,2,4,0.5,0.1,0.3,4,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,50,1,2,5,0.5,0.1,0.3,6,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,50,8,2,196,0.5,0.1,0.3,5,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1596,50,8,2,197,0.5,0.1,0.3,6,3,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1597,50,8,2,198,0.5,0.1,0.3,5,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1598,50,8,2,199,0.5,0.1,0.3,5,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df.columns

Index(['Participant_ID', 'Block Number', 'Session Number', 'Trial Number',
       'Stay_Probability(Colour)', 'Stay_Probability(Texture)',
       'Stay_Probability(Shape)', 'Slow_stim', 'Medium_stim', 'Fast_stim',
       'Colour_stim', 'Texture_stim', 'Shape_stim', 'Choice Trial Index',
       'Colour_distractor', 'Texture_distractor', 'Shape_distractor',
       'Slow_distractor', 'Medium_distractor', 'Fast_distractor',
       'Distractor_consistent_colour', 'Distractor_consistent_texture',
       'Distractor_consistent_shape', 'Distractor_consistent_slow',
       'Distractor_consistent_medium', 'Distractor_consistent_fast',
       'Distractor_shares_colour', 'Distractor_shares_texture',
       'Distractor_shares_shape', 'Distractor_shares_slow',
       'Distractor_shares_medium', 'Distractor_shares_fast'],
      dtype='object')

<p>Check if last choice trial in every block is indeed a choice trial</p>

<h3>Creating a json file with the list of the folders in the design file folder</h3>

In [22]:
import os
import json

In [24]:
designFileList = os.listdir("../Experiment (No Backend)/public/DesignFiles")
designFileList.remove("DesignFileList.json")

with open("../Experiment (No Backend)/public/DesignFiles/DesignFileList.json", "w") as jsonFile:
    json.dump(designFileList, jsonFile)

